# End-to-end demo with MTRAG benchmark data

This notebook shows several examples of end-to-end RAG use cases that use the retrieval
IO processor in conjunction with the IO processors for other Granite-based LoRA 
adapters. More information about the models used here can be found in our [technical
report](https://arxiv.org/html/2504.11704v1).

This notebook can run its own vLLM server to perform inference, or you can host the 
models on your own server. To use your own server, set the `run_server` variable below
to `False` and set appropriate values for the constants in the cell marked
`# Constants go here`.

In [1]:
# Imports go in this cell
import json
import openai

from IPython.display import display, Markdown

import granite_common
from granite_common.base.types import (
    AssistantMessage,
    ChatCompletion,
    ChatCompletionResponse,
    ChatCompletionResponseChoice,
    UserMessage,
    VLLMExtraBody,
)
from granite_common.retrievers.elasticsearch import ElasticsearchRetriever

In [ ]:
# Constants go here
CORPUS_NAMES_MAPPINGS = {
    "govt": "mt-rag-govt-elser-512-100-20240611",
}

elasticsearch_host = "https://localhost:32765"
corpus_name = "govt"
base_model_name = "ibm-granite/granite-3.3-8b-instruct"

openai_base_url = "http://localhost:55555/v1"
openai_api_key = "rag_intrinsics_1234"

query_rewrite_intrinsic_name = "query_rewrite"
citations_intrinsic_name = "citations"
answerability_intrinsic_name = "answerability"
hallucination_detection_intrinsic_name = "hallucination_detection"
uncertainty_intrinsic_name = "uncertainty"

intrinsic_names = [
    query_rewrite_intrinsic_name,
    citations_intrinsic_name,
    answerability_intrinsic_name,
    hallucination_detection_intrinsic_name,
    uncertainty_intrinsic_name,
]

In [3]:
intrinsic_rewriters = {}
intrinsic_result_processors = {}
for intrinsic_name in intrinsic_names:
    io_yaml_file = granite_common.intrinsics.util.obtain_io_yaml(
        intrinsic_name, base_model_name
    )

    intrinsic_rewriter = granite_common.IntrinsicsRewriter(config_file=io_yaml_file)
    intrinsic_result_processor = granite_common.IntrinsicsResultProcessor(
        config_file=io_yaml_file
    )

    intrinsic_rewriters[intrinsic_name] = intrinsic_rewriter
    intrinsic_result_processors[intrinsic_name] = intrinsic_result_processor

# Connect to the inference server
client = openai.OpenAI(base_url=openai_base_url, api_key=openai_api_key)

In [4]:
# Connect to the Elser server
retrievers = {}
for name, corpus_name in CORPUS_NAMES_MAPPINGS.items():
    retriever = ElasticsearchRetriever(
        corpus_name=corpus_name,
        host=elasticsearch_host,
        verify_certs=False,
        ssl_show_warn=False,
    )
    retrievers[name] = retriever

We start by creating an example chat completion request. 

This chat completion request simulates a scenario where the user is chatting with the automated help desk agent of the California State Parks and is asking about internship opportunities. The agent is about to respond to the user's question, "Cool, how to I sign up?"

In [5]:
# Create an example chat completion with a user question and two documents.
chat_input = ChatCompletion.model_validate(
    {
        "messages": [
            {
                "role": "assistant",
                "content": "Welcome to the California State Parks help desk.",
            },
            {
                "role": "user",
                "content": "I'm a student. Do you have internships?",
            },
            {
                "role": "assistant",
                "content": "The California State Parks hires Student Assistants "
                "to perform a variety of tasks that require limited or no previous "
                "work experience.",
            },
            {"role": "user", "content": "Cool, how do I sign up?"},
        ],
        "temperature": 0.0,
        "max_tokens": 4096,
    }
)
print(chat_input.model_dump_json(indent=2))

{
  "messages": [
    {
      "content": "Welcome to the California State Parks help desk.",
      "role": "assistant"
    },
    {
      "content": "I'm a student. Do you have internships?",
      "role": "user"
    },
    {
      "content": "The California State Parks hires Student Assistants to perform a variety of tasks that require limited or no previous work experience.",
      "role": "assistant"
    },
    {
      "content": "Cool, how do I sign up?",
      "role": "user"
    }
  ],
  "temperature": 0.0,
  "max_tokens": 4096
}


Let's start by passing the chat completion request directly to the language model, without using retrieval-augmented generation.

In [6]:
# Pass the example through Granite to get an answer
chat_input.model = base_model_name
non_rag_completion = client.chat.completions.create(**chat_input.model_dump())

display(Markdown(non_rag_completion.choices[0].message.content))

To apply for a Student Assistant position with California State Parks, follow these steps:

1. **Check for Openings**: Visit the California Job Portal (https://www.caljob.ca.gov/) and search for "Student Assistant" or related positions under the "California State Parks" employer.

2. **Create an Account**: If you don't already have one, create an account on the California Job Portal.

3. **Apply**: Once you find a suitable position, click on the job title, review the job description and requirements, and then apply online. You'll need to submit your resume and possibly cover letter.

4. **Wait for Review**: After submitting your application, wait for a response. If your qualifications match the requirements, you may be contacted for an interview.

5. **Interview**: If selected, you'll likely be invited for an interview. This could be in-person or virtual, depending on the current protocols.

6. **Hiring Process**: If you're successful in the interview, you'll go through the hiring process, which may include background checks and reference verifications.

Remember, these positions are often seasonal or part-time and may not be available year-round. It's also advisable to check directly with California State Parks or your local park districts as they might have additional or separate internship programs.

This result is a hallucination. The actual correct answer can be found [here](
    https://www.parks.ca.gov/?page_id=848
).

We can use the 
[Granite 3.3 8b Instruct - Uncertainty LoRA](
    https://huggingface.co/ibm-granite/granite-3.3-8b-rag-agent-lib/blob/main/certainty_lora/README.md)
LoRA adapter to flag cases such as this one that are not covered by the base model's 
training data.

In [7]:
uncertainty_input = chat_input.model_copy(deep=True)
uncertainty_input.model = uncertainty_intrinsic_name

intrinsics_request = intrinsic_rewriters[uncertainty_intrinsic_name].transform(
    uncertainty_input
)

uncertainty_completion = client.chat.completions.create(
    **intrinsics_request.model_dump()
)

uncertainty_score = 1 - float(uncertainty_completion.choices[0].message.content)

print(f"Certainty score is {uncertainty_score} out of 1.0")

Certainty score is 0.0 out of 1.0


The low certainty score indicates that the model's training data does not align closely with this question.

To answer this question properly, we need to provide the model with domain-specific information. In this case, the relevant information can be found in the Government corpus of the [MTRAG multi-turn RAG benchmark](https://github.com/IBM/mt-rag-benchmark).

In [8]:
retriever = retrievers["govt"]

We can send string queries against this vector database to retrieve relevant documents. Here we query the database with the user's last turn from our example conversation, "Cool, how do I sign up?"

In [9]:
# The vector database fetches document snippets that match a given query.
# For example, the user's question in the conversation above:
print(f"Query is: '{chat_input.messages[-1].content}'")
print("Matching document snippets:")
documents = retriever.retrieve(chat_input.messages[-1].content, top_k=3)
documents

Query is: 'Cool, how do I sign up?'
Matching document snippets:


[Document(text="Conditions of Use - CalPERS\nIf you haven't registered for myCalPERS, select the Participant option and Continue, followed by Register Now. You'll need to create a username and password to obtain secure online access to your CalPERS account information.\nYour Password\nYour password is your personal, confidential key to accessing CalPERS information online. Your password and your username are both Secure Socket Layer (SSL) encrypted. During the online access registration and log in processes, we request information about you so we can authenticate who you say you are.\nYou play an important role in making sure your confidential information is secure. To ensure your personal password remains secure, you should never reveal it to anyone. If you believe someone has gained access to your password, you should change it immediately.\nWhen you log in to myCalPERS always verify that your personal security icon and personal security message are the ones you chose. These are know

If we attach a GraniteIO RetrievalRequestProcessor to our vector database, we can use this RequestProcessor to augment the original chat completion request with the document snippets that the retriever fetches when fed the last user turn as a query.

In [10]:
# TODO: Create processor to attach to the extra_body
chat_input_with_docs = chat_input.model_copy(deep=True)
chat_input_with_docs.extra_body = VLLMExtraBody(documents=documents)
chat_input_with_docs.model_dump()

{'messages': [{'content': 'Welcome to the California State Parks help desk.',
   'role': 'assistant'},
  {'content': "I'm a student. Do you have internships?", 'role': 'user'},
  {'content': 'The California State Parks hires Student Assistants to perform a variety of tasks that require limited or no previous work experience.',
   'role': 'assistant'},
  {'content': 'Cool, how do I sign up?', 'role': 'user'}],
 'model': 'ibm-granite/granite-3.3-8b-instruct',
 'extra_body': {'documents': [{'text': "Conditions of Use - CalPERS\nIf you haven't registered for myCalPERS, select the Participant option and Continue, followed by Register Now. You'll need to create a username and password to obtain secure online access to your CalPERS account information.\nYour Password\nYour password is your personal, confidential key to accessing CalPERS information online. Your password and your username are both Secure Socket Layer (SSL) encrypted. During the online access registration and log in processes, 

Note that the retriever here operates over the last user turn. In this particular conversation, the last user turn is the phrase, "Cool, how do I sign up?", which is missing crucial information for retrieving relevant documents -- specifically, what isthe user attempting to sign up for? 

As a result, the snippets retrieved are not specific to the user's intended question. Instead, they cover the general topic of signing up for things.

Let's see what happens if we run our request through the model using these low-quality document snippets.

In [11]:
rag_completion = client.chat.completions.create(**chat_input_with_docs.model_dump())
display(Markdown(rag_completion.choices[0].message.content))

I am sorry, the question is unanswerable from the available information. The provided document does not contain information about applying for internships or student assistant positions at California State Parks.

In this case, the model correctly refuses to answer the question.

Unfortunately, the training data for most LLMs is biased against 
producing this type of result, leading to frequent hallucinations in the presence of faulty retrieved documents. For example, if the last user turn in our example conversation is "How to I sign up?", instead of "*Cool,* how do I sign up?", the model produces an entirely different response:

In [12]:
# Change the last user turn from "Cool, how do I sign up?" to "How to I sign up?"
messages_no_cool = chat_input_with_docs.messages.copy()
messages_no_cool[-1].content = "How do I sign up?"
chat_input_no_cool = chat_input_with_docs.model_copy(
    update={"messages": messages_no_cool}
)
rag_result_no_cool = client.chat.completions.create(**chat_input_no_cool.model_dump())
display(Markdown(rag_result_no_cool.choices[0].message.content))

To sign up for the CalPERS email newsletter, you need to register for a myCalPERS account. Make sure to use a current and correctly spelled email address during registration. Once your email address is registered, you will receive the next Member News email newsletter, typically sent the first Tuesday of every month.

The [LoRA Adapter for Answerability Classification](
    https://huggingface.co/ibm-granite/granite-3.3-8b-rag-agent-lib/blob/main/answerability_prediction_lora/README.md
)
provides a more robust way to detect this kind of problem. Here's what happens if we run the chat completion request with faulty documents snippets through the answerability model, using the `granite_common` IO processor for the model to handle input and output:

In [13]:
answerability_input = chat_input_with_docs.model_copy(deep=True)
answerability_input.model = answerability_intrinsic_name

answerability_request = intrinsic_rewriters[answerability_intrinsic_name].transform(
    answerability_input
)
answerability_completion = client.chat.completions.create(
    **answerability_request.model_dump()
)

answerability = json.loads(answerability_completion.choices[0].message.content)
print(answerability)

unanswerable


The answerability model detects that the documents we have retrieved cannot be used to answer the user's question. We use use a composite IO processor to wrap this check in a flow that falls back on canned response.

In [14]:
DEFAULT_CANNED_RESPONSE = (
    "Sorry, but I am unable to answer this question from the documents retrieved."
)
if answerability == "answerable":
    rag_completion = client.chat.completions.create(**chat_input_with_docs.model_dump())
else:
    rag_completion = ChatCompletionResponse(
        choices=[
            ChatCompletionResponseChoice(
                index=0, message=AssistantMessage(content=DEFAULT_CANNED_RESPONSE)
            )
        ]
    )
display(Markdown(rag_completion.choices[0].message.content))

Sorry, but I am unable to answer this question from the documents retrieved.

At this point we've improved our model output from a hallucinated response to a refusal to answer the question. This result is an improvement, but we can do better if we can retrieve document snippets that are relevant to the user's intent as expressed in the *entire* conversation, not just the last turn.

We can use use the [LoRA Adapter for Query Rewrite](
    https://huggingface.co/ibm-granite/granite-3.2-8b-lora-rag-query-rewrite
) to rewrite the last user turn into a string that is more useful for retrieving document snippets.
Here's what we get if we call this model directly on the original request:

In [15]:
qr_input = chat_input_with_docs.model_copy(deep=True)
qr_input.model = query_rewrite_intrinsic_name

qr_request = intrinsic_rewriters[query_rewrite_intrinsic_name].transform(qr_input)
qr_completion = client.chat.completions.create(**qr_request.model_dump())
print(json.loads(qr_completion.choices[0].message.content)["rewritten_question"])

How do I sign up for the CalPERS email newsletter?


Since the LoRA Adapter for Query Rewrite is a language model, we can ask it to generate multiple different rewrites. We'll use this capability later on to improve end-to-end result quality further.

In [16]:
qr_input.model = query_rewrite_intrinsic_name
qr_input.n = 10
qr_input.temperature = 0.8

qr_request = intrinsic_rewriters[query_rewrite_intrinsic_name].transform(qr_input)
qr_completion = client.chat.completions.create(**qr_request.model_dump())

for choice in qr_completion.choices:
    print(json.loads(choice.message.content)["rewritten_question"])

How do I sign up for the CalPERS Email Newsletter?
How do I register for myCalPERS and access my CalPERS account information?
How do I sign up for the CalPERS Email Newsletter?
How do I sign up for the Calf.State Parks Student Assistant Program?
How do I sign up for the CalPERS email newsletter?
How can I sign up for the CalPERS email newsletter?
How do I sign up for the CalPERS email newsletter?
How can I sign up for the CalPERS email newsletter?
How do I sign up for the CalPERS Email Newsletter?
How do I sign up for the CalPERS email newsletter?


We can wrap the IO processor for this model in a request processor that rewrites
the last turn of the chat completion request.

In [17]:
qr_input = chat_input.model_copy(deep=True)
qr_input.model = query_rewrite_intrinsic_name

qr_request = intrinsic_rewriters[query_rewrite_intrinsic_name].transform(qr_input)
qr_completion = client.chat.completions.create(**qr_request.model_dump())

rewritten_chat_input = chat_input.model_copy(deep=True)
rewritten_chat_input.messages[-1] = UserMessage(
    content=json.loads(qr_completion.choices[0].message.content)["rewritten_question"]
)

print("Messages after rewrite:")
[{"role": m.role, "content": m.content} for m in rewritten_chat_input.messages]

Messages after rewrite:


[{'role': 'assistant',
  'content': 'Welcome to the California State Parks help desk.'},
 {'role': 'user', 'content': "I'm a student. Do you have internships?"},
 {'role': 'assistant',
  'content': 'The California State Parks hires Student Assistants to perform a variety of tasks that require limited or no previous work experience.'},
 {'role': 'user',
  'content': 'How do I sign up for the Student Assistant internship at California State Parks?'}]

We can fetch documents with the rewritten query, then use use the answerability IO processor to check that the fetched documents answer the rewritten query.

In [18]:
documents = retriever.retrieve(rewritten_chat_input.messages[-1].content, top_k=3)

rewritten_chat_input_with_docs = rewritten_chat_input.model_copy(deep=True)
rewritten_chat_input_with_docs.extra_body = VLLMExtraBody(documents=documents)

answerability_input = rewritten_chat_input_with_docs.model_copy(deep=True)
answerability_input.model = answerability_intrinsic_name

answerability_request = intrinsic_rewriters[answerability_intrinsic_name].transform(
    answerability_input
)
answerability_completion = client.chat.completions.create(
    **answerability_request.model_dump()
)
print(json.loads(answerability_completion.choices[0].message.content))

answerable


We can also verify that the fetched documents answer the *original* query prior to the rewrite.

In [19]:
chat_input_with_docs_from_rewrite = chat_input_with_docs.model_copy(deep=True)
chat_input_with_docs_from_rewrite.extra_body.documents = (
    rewritten_chat_input_with_docs.extra_body.documents
)

answerability_input = chat_input_with_docs_from_rewrite.model_copy(deep=True)
answerability_input.model = answerability_intrinsic_name

answerability_request = intrinsic_rewriters[answerability_intrinsic_name].transform(
    answerability_input
)
answerability_completion = client.chat.completions.create(
    **answerability_request.model_dump()
)
print(json.loads(answerability_completion.choices[0].message.content))

answerable


We can chain all of these request processors together with the IO processor for 
the answerability model to create a single flow that processes requests in multiple
steps:
1. Rewrite the last user message for retrieval
1. Retrieve documents and attach them to the request
1. Check for answerability with the retrieved documents
1. If the answerability check passes, then send the request to the base model


In [20]:
# 1. Rewrite
qr_input = chat_input.model_copy(deep=True)
qr_input.model = query_rewrite_intrinsic_name

qr_request = intrinsic_rewriters[query_rewrite_intrinsic_name].transform(qr_input)
qr_completion = client.chat.completions.create(**qr_request.model_dump())
rewritten_question = json.loads(qr_completion.choices[0].message.content)[
    "rewritten_question"
]

new_chat_input = chat_input.model_copy(deep=True)
new_chat_input.messages[-1] = UserMessage(content=rewritten_question)

# 2. Retrieve
documents = retriever.retrieve(new_chat_input.messages[-1].content, top_k=3)
new_chat_input.extra_body = VLLMExtraBody(documents=documents)

# 3. Answerability
answerability_input = new_chat_input.model_copy(deep=True)
answerability_input.model = answerability_intrinsic_name

answerability_request = intrinsic_rewriters[answerability_intrinsic_name].transform(
    answerability_input
)
answerability_completion = client.chat.completions.create(
    **answerability_request.model_dump()
)

# 4. Answerable -> base model to generate
answerability = json.loads(answerability_completion.choices[0].message.content)
DEFAULT_CANNED_RESPONSE = (
    "Sorry, but I am unable to answer this question from the documents retrieved."
)
if answerability == "answerable":
    rag_completion = client.chat.completions.create(**new_chat_input.model_dump())
else:
    rag_completion = ChatCompletionResponse(
        choices=[
            ChatCompletionResponseChoice(
                index=0, message=AssistantMessage(content=DEFAULT_CANNED_RESPONSE)
            )
        ]
    )
display(Markdown(rag_completion.choices[0].message.content))

Since the recruiting and hiring of Student Assistants is decentralized at California State Parks, you must contact individual District and Division Offices to find out if they are recruiting for Student Assistants. For more information, you can reach out to the Student Employment Program Coordinator (Personnel) at California State Parks, P.O {"document_id": "c0fc4497bd2df083-7868-10399"}. Box 942896, Sacramento, CA 94296-0001.

Unlike the responses we've seen so far, this response provides information that is both relevant to the user's intended question and grounded in documents retrieved from the  corpus.

We can use the [LoRA Adapter for Citation Generation](https://huggingface.co/ibm-granite/granite-3.3-8b-rag-agent-lib/blob/main/citation_generation_lora/README.md
) to explain exactly how this response is grounded in the documents that the rewritten user query retrieves.

In [21]:
chat_input_citations = new_chat_input.model_copy(deep=True)
chat_input_citations.model = base_model_name

chat_completion = client.chat.completions.create(**chat_input_citations.model_dump())
chat_input_citations.messages.append(chat_completion.choices[0].message)

citations_input = chat_input_citations.model_copy(deep=True)
citations_input.model = citations_intrinsic_name

citations_request = intrinsic_rewriters[citations_intrinsic_name].transform(
    citations_input
)
citations_completion = client.chat.completions.create(**citations_request.model_dump())

processed_chat_completion = intrinsic_result_processors[
    citations_intrinsic_name
].transform(citations_completion, citations_request)

print("Assistant response:")
display(Markdown(citations_input.messages[-1].content))
print("Citations:")
print(
    json.dumps(
        json.loads(processed_chat_completion.choices[0].message.content), indent=2
    )
)

/proj/dmfexp/8cc/kndtran/github/ibm-granite/granite-common/src/granite_common/base/types.py:42: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `UserMessage` - serialized value may not be as expected [field_name='messages', input_value=ChatCompletionMessage(con... reasoning_content=None), input_type=ChatCompletionMessage])
  PydanticSerializationUnexpectedValue(Expected `AssistantMessage` - serialized value may not be as expected [field_name='messages', input_value=ChatCompletionMessage(con... reasoning_content=None), input_type=ChatCompletionMessage])
  PydanticSerializationUnexpectedValue(Expected `ToolResultMessage` - serialized value may not be as expected [field_name='messages', input_value=ChatCompletionMessage(con... reasoning_content=None), input_type=ChatCompletionMessage])
  PydanticSerializationUnexpectedValue(Expected `SystemMessage` - serialized value may not be as expected [field_name='messages', input_value=ChatCompletionMessage

Assistant response:


<r0> Since the recruiting and hiring of Student Assistants is decentralized at California State Parks, you must contact individual District and Division Offices to find out if they are recruiting for Student Assistants. <r1> For more information, you can reach out to the Student Employment Program Coordinator (Personnel) at California State Parks, P.O {"document_id": "c0fc4497bd2df083-7868-10399"}. <r2> Box 942896, Sacramento, CA 94296-0001.

Citations:
[
  {
    "response_begin": 0,
    "response_end": 220,
    "response_text": "Since the recruiting and hiring of Student Assistants is decentralized at California State Parks, you must contact individual District and Division Offices to find out if they are recruiting for Student Assistants. ",
    "citation_doc_id": "d0067688bcefbff9-0-3492",
    "citation_begin": 13779,
    "citation_end": 16582,
    "citation_text": "Since the recruiting and hiring of Student Assistants is decentralized at California State Parks,\u00a0you must contact individual District and Division Offices to find out if they are recruiting for Student Assistants. "
  },
  {
    "response_begin": 220,
    "response_end": 622,
    "response_text": "For more information, you can reach out to the Student Employment Program Coordinator (Personnel) at California State Parks, P.O {\"document_id\": \"c0fc4497bd2df083-7868-10399\"}. ",
    "citation_doc_id": "d0067688bcefbff9-0-3492",
    "citation_begin": 1658

We can also use the [LoRA Adapter for Hallucination Detection in RAG outputs](
    https://huggingface.co/ibm-granite/granite-3.3-8b-rag-agent-lib/blob/main/hallucination_detection_lora/README.md
) to further verify that each sentence of the assistant response is consistent with the information in the retrieved documents.

In [22]:
chat_input_hallucinations = new_chat_input.model_copy(deep=True)
chat_input_hallucinations.model = base_model_name

chat_completion = client.chat.completions.create(
    **chat_input_hallucinations.model_dump()
)
chat_input_hallucinations.messages.append(chat_completion.choices[0].message)

hallucinations_input = chat_input_hallucinations.model_copy(deep=True)
hallucinations_input.model = hallucination_detection_intrinsic_name

hallucinations_request = intrinsic_rewriters[
    hallucination_detection_intrinsic_name
].transform(hallucinations_input)
hallucinations_completion = client.chat.completions.create(
    **hallucinations_request.model_dump()
)

processed_chat_completion = intrinsic_result_processors[
    hallucination_detection_intrinsic_name
].transform(hallucinations_completion, hallucinations_request)

print("Assistant response:")
display(Markdown(hallucinations_input.messages[-1].content))
print("Hallucination Checks:")
print(
    json.dumps(
        json.loads(processed_chat_completion.choices[0].message.content), indent=2
    )
)

Assistant response:


<i0> Since the recruiting and hiring of Student Assistants is decentralized at California State Parks, you must contact individual District and Division Offices to find out if they are recruiting for Student Assistants. <i1> For more information, you can reach out to the Student Employment Program Coordinator (Personnel) at California State Parks, P.O {"document_id": "c0fc4497bd2df083-7868-10399"}. <i2> Box 942896, Sacramento, CA 94296-0001.

Hallucination Checks:
[
  {
    "response_begin": 0,
    "response_end": 220,
    "response_text": "Since the recruiting and hiring of Student Assistants is decentralized at California State Parks, you must contact individual District and Division Offices to find out if they are recruiting for Student Assistants. ",
    "faithfulness_likelihood": 0.9734434613847318,
    "explanation": "This sentence makes a factual claim about the decentralized nature of recruiting and hiring at California State Parks. The document states 'Since the recruiting and hiring of Student Assistants is decentralized at California State Parks, you must contact individual District and Division Offices to find out if they are recruiting for Student Assistants.' This matches exactly with the claim in the sentence."
  },
  {
    "response_begin": 220,
    "response_end": 622,
    "response_text": "For more information, you can reach out to the Student Employment Program Coordinator (Personnel) at California State 

### TODO: Add composite IO processor